# Modélisation 

In [76]:
# Functions imports

import tqdm as notebook_tqdm

import importlib
import functions
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV

importlib.reload(functions)
from functions import *

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [77]:
x = pd.read_csv("../data/processed/x_processed.csv")  # 10605 rows × 10 columns
x_challenge = pd.read_csv("../data/processed/x_challenge_processed.csv")  # 4942 rows × 10 columns
y = pd.read_csv("../data/raw/y_train_jJtXgMX.csv")  # 10605 rows × 2 columns

Préparation des données 

In [78]:
y = y.drop(columns=['DELIVERY_START'], axis=1)
x = x.drop(columns=['DELIVERY_START'], axis=1)

y['target'] = (y['spot_id_delta'] > 0).astype(int)

y = y.drop(columns=['spot_id_delta'], axis=1)

y.head(15)

,target
0,0
1,0
2,0
3,1
4,1
5,0
6,0
7,0
8,0
9,0


In [79]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3)

In [80]:
# Normalisation 

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

x_train = pd.DataFrame(x_train) # scaler renvoie des numpy.darray
x_test = pd.DataFrame(x_test)

In [81]:
forest = RandomForestClassifier()

cross_validation(forest, x_train, y_train['target'], 5, 'accuracy')

##### CROSS-VALIDATION SCORES ##### 

 accuracy  scores from cross-validation  [0.78922559 0.79461279 0.79191919 0.80592992 0.80121294] 
 accuracy  mean from cross-validation  0.7965800865800866 
 Standard deviation  0.0061410990749509375 

 ##################################


# Optimisation

In [82]:
# On commence par une Randomized Search, on passera à un HyperOpt plus tard s'il faut car pour l'instant
# j'ai des problèmes avec l'importation d'un package

# Paramètres pour la RandomizedSearchCV

param_dist = {
    'n_estimators': [int(x) for x in np.linspace(start=100, stop=1200, num=12)],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [int(x) for x in np.linspace(10, 110, num=11)] + [None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

forest_random = RandomizedSearchCV(
    estimator=forest,
    param_distributions=param_dist,
    n_iter=100,
    cv=3,
    verbose=2,
    n_jobs=-1
)

forest_random.fit(x_train, y_train['target'])

print(f"Meilleurs paramètres : ", {forest_random.best_params_})



Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=1200; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=1200; total time=   0.0s
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1100; total time=   0.0s
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1100; total time=   0.0s
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1100; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=1200; total time=   0.0s
[CV] END bootstrap=True, max_depth=90, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimat

/Users/hugopuybareau/Documents/GitHub/ELMY_DATA_CHALLENGE/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
117 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/hugopuybareau/Documents/GitHub/ELMY_DATA_CHALLENGE/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/hugopuybareau/Documents/GitHub/ELMY_DATA_CHALLENGE/.venv/lib/python3.12/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/Users/hugopuybareau/Docum

TypeError: unhashable type: 'dict'